# Three Market Database Demo

In [ ]:
import passengersim as pax

In [ ]:
input_file = pax.demo_network("3mkt")

In this notebook, we will demonstrate the usage of SQLite database logging.  In order 
to manipulate the configurations, we'll separate the simulation initialization into
two steps: first we load the configs from YAML, then we apply configs to a Simlaution.
That way, we can maipulate those configs directly in Python before running the sim.

So, first we load the configs.

In [ ]:
cfg = pax.Config.from_yaml(input_file)

Then, we can manipulate the configs.  By default we our database engine will store detailed info on 
multiple simulation attributes at each DCP:

In [ ]:
cfg.db.write_items

Let's remove the "demand_final" writer, so we can replace it with a custom tool we provide in Python.

In [ ]:
cfg.db.write_items.remove("demand_final")
cfg.db.write_items

Our custom Python command is `save_demand_to_database`, in the `passengersim.database.write_demands` package.
We can attach that function to the list of `dcp_write_hooks`.

In [ ]:
from passengersim.database.write_demands import save_demand_to_database

cfg.db.dcp_write_hooks.append(save_demand_to_database)

We can also set our SQLite database engine to reside exclusively in memory, like this:

In [ ]:
cfg.db.filename = ":memory:"

In [ ]:
import os
if os.path.exists(cfg.db.filename):
    os.remove(cfg.db.filename)

We use the compiled code for writing SQL with the `fast` flag:

In [ ]:
cfg.db.fast = True

Some experimental "pragma" commands that may make SQLite go faster...

In [ ]:
cfg.db.pragmas = [
    "journal_mode = MEMORY",
    "synchronous = 0",
    "cache_size = 1000000",
    "locking_mode = EXCLUSIVE",
    "temp_store = MEMORY",
]

We don't want to *also* write raw data files, so we'll turn those off:

In [ ]:
cfg.simulation_controls.write_raw_files = False

Now we are ready to create and run our simulation.

In [ ]:
sim = pax.Simulation(cfg, output_dir="./demo-3mkt-out")

In [ ]:
summary = sim.run(log_reports=False)

In [ ]:
summary.demands

In [ ]:
summary.legs

In [ ]:
summary.paths

In [ ]:
summary.airlines

In [ ]:
summary.load_factors

In [ ]:
summary.bookings_by_timeframe

In [ ]:
summary.fig_bookings_by_timeframe()

In [ ]:
summary.fig_bookings_by_timeframe(by_airline=False)

In [ ]:
summary.fig_carrier_loads(raw_df=True)

In [ ]:
summary.fig_fare_class_mix()

In [ ]:
summary.total_demand

We've written only to an in-memory database, but we want to dump that thing to disk now for posterity:

In [ ]:
sim.backup_db("backup-db.sqlite")

We can get dataframes from SQL queries while we have the database connection open...

In [ ]:
bookings = sim.cnx.dataframe("SELECT * FROM bookings_by_timeframe")
bookings

Once we close the database connection, we can no longer use it for reading or writing.  
If it was an in-memory database, closing it means that content is gone for good, unless it
was backed up to disk (as we did above).

In [ ]:
sim.cnx.close()

In [ ]:
leg_detail = sim.cnx.dataframe("SELECT * FROM leg_detail")